In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pingouin as pg
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from os.path import join as pjoin
from scipy.stats import zscore, norm
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys
import re
from tqdm import tqdm
import pymannkendall as mk

In [ ]:
sys.path.append('..')
import circletrack_behavior as ctb
import plotting_functions as pf
import pca_ica as ica 

In [ ]:
behav_path_cohort0 = '/media/caishuman/csstorage/phild/git/MazeProjects/output/behav'
behav_path_cohort1 = '../../EnsembleRemodeling_Resubmission/circletrack_data/output/behav' 
ensemble_path_cohort0 = '/media/caishuman/csstorage/phild/git/MazeProjects/output/assemblies'
ensemble_path_cohort1 = '../../EnsembleRemodeling_Resubmission/circletrack_data/output/assemblies'
mice_cohort0 = ['Fornax', 'Janus', 'Lyra', 'Miranda', 'Naiad', 'Sao', 'Titania']
mice_cohort1 = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']
combined_mice = mice_cohort0 + mice_cohort1
session_list_cohort0 = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1']
session_list_cohort1 = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']
session_dict = {'Fornax': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Janus': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Lyra': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Miranda': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Naiad': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Sao': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'Titania': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1'],
                'mc03': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal4', 'Training_Reversal'],
                'mc06': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc07': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc09': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc11': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']}

In [ ]:
## Set mouse list, path, and z_thresh for binarization
mouse_list = combined_mice
across_time = False
analysis = 'max'
alpha_old = 0.05
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(mouse_list):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    if mouse in mice_cohort0:
        ensemble_path = ensemble_path_cohort0
        behav_path = behav_path_cohort0
        trial_type = 'all'
    elif mouse in mice_cohort1:
        ensemble_path = ensemble_path_cohort1
        behav_path = behav_path_cohort1
        behav_path = pjoin(behav_path, '{}'.format(mouse))
        ensemble_path = pjoin(ensemble_path, '{}'.format(mouse))
        trial_type = 'forward'
    ## Load assemblies
    for session in session_dict[mouse]:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, ensemble_path, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            behavior_df = pd.read_feather(behav_file)
            if mouse in mice_cohort1:
                aligned_behavior = ctb.align_behavior_frames(behavior_df, session='30min', sampling_rate=1/15)
                ## Ensure that activations and aligned behavior are the same length
                aligned_behavior = ica.align_activations_to_behavior(act, aligned_behavior)
            else:
                aligned_behavior = behavior_df
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp
                last_timestamp = last_timestamp
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = analysis_type, 
                                                                                           zscored = True, alpha_old = alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trial_type=trial_type, 
                                                                                               alpha_old=alpha_old, analysis=analysis)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

In [ ]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df['Training4'].mean(), prop_df['Reversal1'].mean()]
sem = [prop_df['Training4'].sem(), prop_df['Reversal1'].sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data['Training4'], data['Reversal1']], mode = 'lines', line_color = 'black', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image(pjoin(figure_path, 'proportion_fading_trials_mean_OLD.png'))

In [ ]:
## Wilcoxon Signed-Rank Test
wilcoxon_result = wilcoxon(x=prop_df['Training4'], y=prop_df['Reversal1'], method='approx')
wilcoxon_result

In [ ]:
## Paired t-test
t_test = pg.ttest(x=prop_df['Training4'], y=prop_df['Reversal1'], paired=True)

In [ ]:
## Test for equal variance between groups (sex)
prop_df.insert(0, 'sex', ['f', 'f', 'f', 'm', 'm', 'f', 'f', 'm', 'm', 'm', 'm', 'm'])
prop_df.insert(1, 'cohort', [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
melt_df = prop_df.melt(id_vars=['sex', 'cohort', 'mouse'])
melt_df = melt_df.loc[melt_df['variable'] == 'Reversal1'] ## select only Reversal1
pg.homoscedasticity(melt_df, dv='value', group='sex')

In [ ]:
## ANOVA looking at sex
prop_df.anova(dv='Reversal1', between='sex', ss_type=3, detailed=True)

In [ ]:
## ANOVA looking at cohort
prop_df.anova(dv='Reversal1', between='cohort', ss_type=3, detailed=True)

In [ ]:
mouse_pc = {}
mouse_pc_trial = {}
mouse_sdt = {}
mouse_rewards = {}
for mouse in combined_mice:
    percent_correct = {}
    percent_correct_trials = {}
    sdt_metrics = {}
    rewards = {}
    ## Change path
    if mouse in mice_cohort0:
        behav_path = behav_path_cohort0
    elif mouse in mice_cohort1:
        behav_path = behav_path_cohort1
        behav_path = pjoin(behav_path, mouse)

    for session in session_dict[mouse]:
        behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
        behav = pd.read_feather(behav_file)
        num_rewards = np.sum(behav['water'])
        rewards[session] = num_rewards
        if mouse in mice_cohort0:
            reward_one, reward_two = np.unique(behav.loc[behav['water'] == True, 'lick_port'])[1], np.unique(behav.loc[behav['water'] == True, 'lick_port'])[2]
            behav = ctb.fix_lick_ports(behav, reward_one, reward_two)
            pc = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, by_trials=False)
            pc_trials = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, by_trials=True)
            percent_correct[session] = pc
            percent_correct_trials[session] = pc_trials
            ## dprime, correct rejection, hit rate
            metrics = ctb.dprime_metrics(behav, reward_one, reward_two, reward_index='zero', forward_reverse='all')
            sdt_metrics[session] = metrics
        
        elif mouse in mice_cohort1:
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            forward_trials, reverse_trials = ctb.get_forward_reverse_trials(behav)
            forward_df = behav.loc[(behav['trials'].isin(forward_trials))]
            ## Percent correct licking
            pc = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=False)
            pc_trials = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=True)
            percent_correct[session] = pc
            percent_correct_trials[session] = pc_trials
            ## dprime, correct rejection, hit rate
            metrics = ctb.dprime_metrics(forward_df, reward_one, reward_two, reward_index='one', forward_reverse='forward')
            sdt_metrics[session] = metrics
            
    mouse_pc[mouse] = percent_correct
    mouse_pc_trial[mouse] = percent_correct_trials
    mouse_sdt[mouse] = sdt_metrics
    mouse_rewards[mouse] = rewards

In [ ]:
## Percent correct data frame for graphing
accuracy = pd.DataFrame()
for mouse in mouse_pc:
        df = pd.DataFrame(mouse_pc[mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
        df.insert(0, 'mouse', mouse)
        df.insert(1, 'session', [x for x in mouse_pc[mouse]])
        accuracy = pd.concat([accuracy, df], axis=0, ignore_index=True)
## Select common sessions between groups
df = accuracy.loc[(accuracy['session'] == 'Training1') | (accuracy['session'] == 'Training2') | 
                  (accuracy['session'] == 'Training3') | (accuracy['session'] == 'Training4') | (accuracy['session'] == 'Reversal1')]
df = df.replace(to_replace={'Training1': 1, 'Training2': 2, 'Training3': 3, 'Training4': 4, 'Reversal1': 5})
fig = pf.plot_behavior_across_days(df, x_var='session', y_var='percent_correct', groupby_var=['session'], plot_transitions=[4.5], title='Lick Accuracy',
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white')
fig.show()

In [ ]:
## d' across days
dprime_df = pd.DataFrame()
for mouse in mouse_sdt:
    df = pd.DataFrame(mouse_sdt[mouse]).T
    d = []
    for idx in np.arange(0, len(df['dprime'])):
        d.append(np.nanmean(df['dprime'][idx]))
    dprime = pd.DataFrame(d).rename(columns={0: 'dprime'})
    dprime.insert(0, 'mouse', mouse)
    dprime.insert(1, 'session', [x for x in mouse_sdt[mouse]])
    dprime_df = pd.concat([dprime_df, dprime], axis=0, ignore_index=True)
## Select common sessions between groups
df = dprime_df.loc[(dprime_df['session'] == 'Training1') | (dprime_df['session'] == 'Training2') | 
                  (dprime_df['session'] == 'Training3') | (dprime_df['session'] == 'Training4') | (dprime_df['session'] == 'Reversal1')]
df = df.replace(to_replace={'Training1': 1, 'Training2': 2, 'Training3': 3, 'Training4': 4, 'Reversal1': 5})

fig = pf.plot_behavior_across_days(df, x_var='session', y_var='dprime', groupby_var=['session'], plot_transitions=[4.5], title='',
                                   x_title='Day', y_title="d'", template='simple_white')
fig.show()

In [ ]:
## Rewards across days
rewards_df = pd.DataFrame()
for mouse in mouse_rewards:
        df = pd.DataFrame(mouse_rewards[mouse], index=[0]).T.rename(columns={0: 'rewards'})
        df.insert(0, 'mouse', mouse)
        df.insert(1, 'session', [x for x in mouse_rewards[mouse]])
        rewards_df = pd.concat([rewards_df, df], axis=0, ignore_index=True)
## Select common sessions between groups
df = rewards_df.loc[(rewards_df['session'] == 'Training1') | (rewards_df['session'] == 'Training2') | 
                  (rewards_df['session'] == 'Training3') | (rewards_df['session'] == 'Training4') | (rewards_df['session'] == 'Reversal1')]
df = df.replace(to_replace={'Training1': 1, 'Training2': 2, 'Training3': 3, 'Training4': 4, 'Reversal1': 5})
fig = pf.plot_behavior_across_days(df, x_var='session', y_var='rewards', groupby_var=['session'], plot_transitions=[4.5], title='Total Rewards',
                                   x_title='Day', y_title='Total Rewards', template='simple_white')
fig.show()